<a href="https://colab.research.google.com/github/swarnim-sawane/test_repositery/blob/master/openllms_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets replicate rouge google-generativeai

In [4]:
import os
import google.generativeai as palm
import pandas as pd
import replicate
from rouge import Rouge
from datasets import load_dataset

os.environ["REPLICATE_API_TOKEN"] = 'Your_auth_token'
palm.configure(api_key='Your_auth_token')

# Dataset

In [5]:
dataset = load_dataset("gopalkalpande/bbc-news-summary")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
train_data = dataset["train"]

data_list = []
for i in train_data:
    data_list.append({
        "File Path": i["File_path"],
        "Article": i["Articles"],
        "Summary": i["Summaries"]
    })
df = pd.DataFrame(data_list)
df

,File Path,Article,Summary
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...
2219,business,India opens skies to competition..India will a...,India will allow domestic commercial airlines ...
2220,business,Yukos bankruptcy 'not US matter'..Russian auth...,Yukos says a US court was entitled to declare ...
2221,business,Survey confirms property slowdown..Government ...,House prices were 11.8% higher on the year in ...
2222,business,High fuel prices hit BA's profits..British Air...,"Rod Eddington, BA's chief executive, said the ..."


In [7]:
df = df.head(5)

# Llama

**API**

In [8]:
llama_summaries = []

for index, row in df.iterrows():
    text = row["Article"]

    output = replicate.run(
        "meta/llama-2-70b:a52e56fee2269a78c9279800ec88898cecb6c8f1df22a6483132bea266648f00",
        max_new_tokens=250,
        min_new_tokens=180,
        temperature=0.5,
        input={"prompt": 'summarize the following news article: ' + text}
    )

    llm_summary = ""
    for item in output:
        llm_summary += item

    llama_summaries.append(llm_summary)

df["llama_Summary"] = llama_summaries

<ipython-input-8-af4847409874>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["llama_Summary"] = llama_summaries


In [9]:
df

,File Path,Article,Summary,llama_Summary
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...,UKIP are calling for a cut in fuel duty and a...
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ...","He added: ""I think we have got to try and fin..."
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...,\n7. A group of 10 students from different cou...
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...,"Mr Bennett said: ""We cannot continue to let m..."
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g...",They say his claim to have discovered an alle...


**Hugging Face**

In [ ]:
# import torch
# import transformers
# from transformers import AutoTokenizer

# model = "meta-llama/Llama-2-7b"
# prompt = "summarize this news article:" + text

# tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

# def get_llama_response(prompt : str) -> None:
#   llama_pipeline = pipeline(
#     prompt,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id,
#     max_length=200,
#   )
#   for seq in sequences:
#     print(f"{seq['generated_text'][0]}")

In [ ]:
# prompt = "summarize this news article: " + text
# get_llama_response(prompt)

# Mistral

**API**

In [10]:
import replicate

mistral_summaries = []

for index, row in df.iterrows():
    text = row["Article"]

    output = replicate.run(
        "mistralai/mistral-7b-instruct-v0.1:83b6a56e7c828e667f21fd596c338fd4f0039b46bcfa18d973e8e70e455fda70",
        max_new_tokens=250,
        min_new_tokens=180,
        temperature=0.5,
        input={"prompt": 'summarize the following news article: ' + text}
    )

    mistral_summary = ""
    for item in output:
        mistral_summary += item

    mistral_summaries.append(mistral_summary)

df["mistral_Summary"] = mistral_summaries

<ipython-input-10-962212df4974>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mistral_Summary"] = mistral_summaries


In [ ]:
df

**Hugging face**

In [11]:
df

,File Path,Article,Summary,llama_Summary,mistral_Summary
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...,UKIP are calling for a cut in fuel duty and a...,"In his ninth Budget, Gordon Brown will focus o..."
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ...","He added: ""I think we have got to try and fin...",The Army Board is expected to make a final dec...
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...,\n7. A group of 10 students from different cou...,"Michael Howard, the leader of the Conservative..."
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...,"Mr Bennett said: ""We cannot continue to let m...",The UK government has announced that it will i...
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g...",They say his claim to have discovered an alle...,"Robert Kilroy-Silk, an ex-chat show host, has ..."


**Hugging Face**

In [ ]:
# !pip3 install huggingface-hub
# huggingface-cli download TheBloke/Mistral-7B-v0.1-GGUF mistral-7b-v0.1.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False
# from ctransformers import AutoModelForCausalLM

# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF", model_file="mistral-7b-v0.1.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)

# print(llm("summarize this news article: " + text))


# Palm

In [16]:
palm_summaries = []

for index, row in df.iterrows():
    text = row["Article"]

    prompt = "summarize the following news article: " + text

    completion = palm.generate_text(
        model='models/text-bison-001',
        prompt=prompt,
        temperature=0.5,
        max_output_tokens=250,
    )


    palm_summary = completion.result

    palm_summaries.append(palm_summary)

df["palm_Summary"] = palm_summaries

<ipython-input-16-88e8fb15cafa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["palm_Summary"] = palm_summaries


In [17]:
df

,File Path,Article,Summary,llama_Summary,mistral_Summary,palm_Summary
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...,UKIP are calling for a cut in fuel duty and a...,"In his ninth Budget, Gordon Brown will focus o...",Chancellor Gordon Brown will announce his nint...
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ...","He added: ""I think we have got to try and fin...",The Army Board is expected to make a final dec...,Military chiefs to meet to make final decision...
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...,\n7. A group of 10 students from different cou...,"Michael Howard, the leader of the Conservative...",Michael Howard has denied that his shadow cabi...
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...,"Mr Bennett said: ""We cannot continue to let m...",The UK government has announced that it will i...,Ministers to invite international observers to...
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g...",They say his claim to have discovered an alle...,"Robert Kilroy-Silk, an ex-chat show host, has ...",- Ex-chat show host Robert Kilroy-Silk is to c...


# Rouge

In [51]:
data = {
    "llama": {
        "Reference_Summary": df["Article"],
        "Generated_Summary": df["llama_Summary"],
    },
    "mistral": {
        "Reference_Summary": df["Article"],
        "Generated_Summary": df["mistral_Summary"],
    },
    "palm": {
        "Reference_Summary": df["Article"],
        "Generated_Summary": df["palm_Summary"],
    },
}

rouge_df = pd.DataFrame()

rouge = Rouge()

columns = []
results = []

for model_name, model_data in data.items():

    rouge_1_scores = []
    rouge_2_scores = []
    rouge_l_scores = []

    for i in range(len(model_data["Reference_Summary"])):
        reference_summary = model_data["Reference_Summary"][i]
        generated_summary = model_data["Generated_Summary"][i]

        scores = rouge.get_scores(generated_summary, reference_summary, avg=True)

        rouge_1_scores.append(scores["rouge-1"]["f"])
        rouge_2_scores.append(scores["rouge-2"]["f"])
        rouge_l_scores.append(scores["rouge-l"]["f"])

    model_columns = [
        f"{model_name}_ROUGE-1",
        f"{model_name}_ROUGE-2",
        f"{model_name}_ROUGE-L"
    ]
    columns.extend(model_columns)
    results.extend([rouge_1_scores, rouge_2_scores, rouge_l_scores])

rouge_df = pd.DataFrame(results).T
rouge_df.columns = columns



In [52]:
rouge_df

,llama_ROUGE-1,llama_ROUGE-2,llama_ROUGE-L,mistral_ROUGE-1,mistral_ROUGE-2,mistral_ROUGE-L,palm_ROUGE-1,palm_ROUGE-2,palm_ROUGE-L
0,0.079545,0.007366,0.062500,0.335079,0.193548,0.324607,0.240437,0.127208,0.234973
1,0.140673,0.019011,0.134557,0.337278,0.233577,0.337278,0.241830,0.156942,0.241830
2,0.106667,0.000000,0.101333,0.392371,0.232945,0.381471,0.448649,0.321725,0.448649
3,0.187668,0.017986,0.166220,0.309859,0.164234,0.298592,0.635071,0.562986,0.635071
4,0.218845,0.032587,0.170213,0.368750,0.216769,0.362500,0.417722,0.323770,0.417722


In [59]:
data = {
    "llama": {
        "ROUGE-1": rouge_df["llama_ROUGE-1"],
        "ROUGE-2": rouge_df["llama_ROUGE-2"],
        "ROUGE-L": rouge_df["llama_ROUGE-L"],
    },
    "mistral": {
        "ROUGE-1": rouge_df["mistral_ROUGE-1"],
        "ROUGE-2": rouge_df["mistral_ROUGE-2"],
        "ROUGE-L": rouge_df["mistral_ROUGE-L"],
    },
    "palm": {
        "ROUGE-1": rouge_df["palm_ROUGE-1"],
        "ROUGE-2": rouge_df["palm_ROUGE-2"],
        "ROUGE-L": rouge_df["palm_ROUGE-L"],
    },
}

def create_model_rouge_df(model_name, data):
    avg_rouge_1 = sum(data["ROUGE-1"]) / len(data["ROUGE-1"])
    avg_rouge_2 = sum(data["ROUGE-2"]) / len(data["ROUGE-2"])
    avg_rouge_l = sum(data["ROUGE-L"]) / len(data["ROUGE-L"])

    model_rouge_scores = {
        model_name: [avg_rouge_1, avg_rouge_2, avg_rouge_l],
    }

    model_rouge_df = pd.DataFrame(model_rouge_scores, index=["ROUGE-1", "ROUGE-2", "ROUGE-L"])

    return model_rouge_df

final_df = pd.DataFrame()

for model_name, model_data in data.items():
    model_rouge_df = create_model_rouge_df(model_name, model_data)


    final_df = pd.concat([final_df, model_rouge_df], axis=1)

final_df

,llama,mistral,palm
ROUGE-1,0.146679,0.348667,0.396742
ROUGE-2,0.015390,0.208215,0.298526
ROUGE-L,0.126964,0.340890,0.395649


In [61]:

average_scores = final_df.mean()
best_model = average_scores.idxmax()
best_scores = average_scores[best_model]


print("Average ROUGE Scores:")
print(average_scores)
print("\nBest Model:")
print(f"The best model is '{best_model}' with an average ROUGE score of {best_scores:.6f}")


Average ROUGE Scores:
llama      0.096345
mistral    0.299257
palm       0.363639
dtype: float64

Best Model:
The best model is 'palm' with an average ROUGE score of 0.363639


# Justification

***We've selected the Palm based on its consistently higher average ROUGE scores, signifying its proficiency in generating summaries that closely align with reference summaries. While other factors play a role, these strong ROUGE scores make Palm the preferred choice for text summarization.***